In [1]:
import importlib
from oceanwave_forecast import mlflow_utils, config
importlib.reload(mlflow_utils)
importlib.reload(config)
from mlflow.tracking import MlflowClient
import mlflow
from urllib.parse import quote
from loguru import logger

def fetch_and_log_runs_structured(experiment_name: str):
    """Fetch and display MLflow runs organized by run_number, using PROJ_ROOT/mlruns."""
    # ——— 1. ensure tracking URI is set to PROJ_ROOT/mlruns ———
    mlruns_path = config.PROJ_ROOT / "mlruns"
    mlruns_path.mkdir(parents=True, exist_ok=True)
    # URL‑encode to handle spaces, windows paths, etc.
    uri = f"file:///{quote(str(mlruns_path.absolute()), safe=':/')}"
    mlflow.set_tracking_uri(uri)
    logger.info(f"MLflow tracking URI set to: {uri}")

    try:
        # ——— 2. locate experiment by name ———
        client = MlflowClient()
        exp = client.get_experiment_by_name(experiment_name)
        if exp is None:
            logger.error(f"Experiment '{experiment_name}' not found at {mlruns_path}.")
            return

        # ——— 3. pull runs as DataFrame ———
        runs_df = mlflow.search_runs(
            experiment_ids=[exp.experiment_id],
            order_by=["attributes.start_time DESC"],
            output_format="pandas",
        )
        if runs_df.empty:
            logger.warning(f"No runs found for experiment '{experiment_name}'.")
            return

        # ——— 4. assemble detailed run info ———
        runs_data = []
        for rid in runs_df["run_id"]:
            run = client.get_run(rid)
            runs_data.append({
                "run_id": rid,
                "params": run.data.params,
                "metrics": run.data.metrics,
                "artifacts": [f.path for f in client.list_artifacts(rid)]
            })
        # sort by the integer run_number param
        runs_data.sort(key=lambda x: int(x["params"]["run_number"]))

        # ——— 5. pretty‑print ———
        print("=" * 60)
        print(f"MLFLOW EXPERIMENT: {experiment_name}")
        print("=" * 60)
        for run_data in runs_data:
            params = run_data["params"]
            metrics = run_data["metrics"]
            artifacts = run_data["artifacts"]
            run_number = params["run_number"]
            model_name = params.get("model_type", params.get("model_name", "Unknown"))

            print(f"\n🔹 RUN {run_number} - {model_name}")
            print("-" * 40)
            print("📋 PARAMETERS:")
            for k, v in params.items():
                if k not in {"run_number", "model_type", "model_name"}:
                    print(f"  • {k}: {v}")
            print("📊 METRICS:")
            if metrics:
                for mk, mv in metrics.items():
                    print(f"  • {mk}: {mv:.6f}")
            else:
                print("  • No metrics recorded")
            print("🖼️ ARTIFACTS:")
            if artifacts:
                for artifact in artifacts:
                    print(f"  • {artifact}")
            else:
                print("  • No artifacts recorded")
            print()

    except MlflowException as e:
        logger.exception(f"MLflow error: {e}")
    except Exception as e:
        logger.exception(f"Unexpected error: {e}")

fetch_and_log_runs_structured(config.MLFLOW_CONFIG_BACKTESTING['experiment_name'])
fetch_and_log_runs_structured(config.MLFLOW_DEEPAR_CONFIG['experiment_name'])

2025-07-28 07:55:13.632 | INFO     | oceanwave_forecast.config:<module>:12 - PROJ_ROOT path is: D:\CML\Term 8\ML projects\forecasting_workspace\oceanwave_forecast
2025-07-28 07:55:13.635 | INFO     | oceanwave_forecast.config:<module>:12 - PROJ_ROOT path is: D:\CML\Term 8\ML projects\forecasting_workspace\oceanwave_forecast
2025-07-28 07:55:13.637 | INFO     | __main__:fetch_and_log_runs_structured:18 - MLflow tracking URI set to: file:///D:%5CCML%5CTerm%208%5CML%20projects%5Cforecasting_workspace%5Coceanwave_forecast%5Cmlruns
2025-07-28 07:55:14.792 | INFO     | __main__:fetch_and_log_runs_structured:18 - MLflow tracking URI set to: file:///D:%5CCML%5CTerm%208%5CML%20projects%5Cforecasting_workspace%5Coceanwave_forecast%5Cmlruns


MLFLOW EXPERIMENT: Oceanwave_Backtesting

🔹 RUN 0 - NaiveForecaster
----------------------------------------
📋 PARAMETERS:
  • forecaster_params: {'forecasters': [('naive_last_WVHT', NaiveForecaster(), [0]), ('naive_mean_APD', NaiveForecaster(strategy='mean'), [1])], 'naive_last_WVHT': NaiveForecaster(), 'naive_mean_APD': NaiveForecaster(strategy='mean'), 'naive_last_WVHT__sp': 1, 'naive_last_WVHT__strategy': 'last', 'naive_last_WVHT__window_length': None, 'naive_mean_APD__sp': 1, 'naive_mean_APD__strategy': 'mean', 'naive_mean_APD__window_length': None}
📊 METRICS:
  • No metrics recorded
🖼️ ARTIFACTS:
  • No artifacts recorded


🔹 RUN 1 - XGBoost
----------------------------------------
📋 PARAMETERS:
  • colsample_bytree: 0.8
  • device: cuda
  • gamma: 0.0
  • learning_rate: 0.05
  • max_depth: 6
  • min_child_weight: 1.0
  • n_estimators: 800
  • n_jobs: -1
  • objective: reg:squarederror
  • random_state: 42
  • reg_alpha: 0.0
  • reg_lambda: 1.0
  • subsample: 0.8
  • targets: WVH